In [4]:
%load_ext blackcellmagic

In [6]:
import geopandas as gpd
import pandas as pd
import os
import re
from shapely.geometry import Point
from fiona.crs import from_epsg
import logging

# configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler("error_log.log")
handler.setLevel(logging.ERROR)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)


path = "/Users/anastasiaclark/MyStaff/Git_Work/MTA-Mass-Transit"
path = os.getcwd()
yearMonth = "Jun2019"
folder = "June2019"


def process_stops(file):
    """Read bus stop tables, joins them and dedups"""
    stops = pd.read_csv(os.path.join(file, "stops.txt"))
    stop_times = pd.read_csv(os.path.join(file, "stop_times.txt"))
    trips = pd.read_csv(os.path.join(file, "trips.txt"))
    stops = pd.merge(stops, stop_times, on="stop_id", how="left")
    stops = stops.merge(trips, on="trip_id", how="left")
    stops = pd.DataFrame(
        stops, columns=["stop_id", "stop_name", "stop_lat", "stop_lon", "route_id"]
    )
    unique_routes_at_stops = stops.drop_duplicates()
    return unique_routes_at_stops


def make_stop_shapes(df, x="stop_lon", y="stop_lat"):
    points = [Point(xy) for xy in zip(df[x], df[y])]
    gdf = gpd.GeoDataFrame(df, geometry=points)
    gdf.crs = from_epsg(4269)  # initiate crs as NAD83
    gdf = gdf.to_crs(epsg=2263)  # NY State Plane
    return gdf


def make_bus_stops(path, folder):
    buses = ["mn_bus", "si_bus", "qn_bus", "bx_bus", "bk_bus", "bus_company"]
    bus_stops = []
    try:
        for bus in buses:
            file = os.path.join(path, folder, f"{bus}")
            stops = process_stops(file)
            bus_stops.append(stops)

        all_stops = pd.concat(bus_stops)

        local_stops_mask = all_stops["route_id"].str.match(r"\w\d+|BX\d+")
        local_stops_mask = local_stops_mask.fillna(False)

        local_stops = all_stops.loc[local_stops_mask].copy()
        express_stops = all_stops.loc[~local_stops_mask].copy()

        local_stop_shapes = make_stop_shapes(local_stops)
        express_stop_shapes = make_stop_shapes(express_stops)

        counties = gpd.read_file(
            os.path.join(path, "counties_bndry.geojson"), driver="GeoJSON"
        )
        counties = counties.to_crs(from_epsg(2263))

        local_stop_shapes = gpd.sjoin(
            local_stop_shapes, counties, how="inner", op="intersects"
        ).drop(["route_id", "index_right"], 1)

        express_stop_shapes = gpd.sjoin(
            express_stop_shapes, counties, how="inner", op="intersects"
        ).drop(["route_id", "index_right"], 1)

        local_stop_shapes.to_file(
            os.path.join(path, folder, "shapes", f"bus_stops_nyc_{yearMonth}.shp")
        )

        express_stop_shapes.to_file(
            os.path.join(
                path, folder, "shapes", f"express_bus_stops_nyc_{yearMonth}.shp"
            )
        )
        print (f"Created stop shapefiles for local and express bus stops")
        
    except Exception as e:
        logger.exception("Unexpected exception occurred")
        raise

In [7]:
make_bus_stops(path=path, folder=folder)

Created stop shapefiles for local and express bus stops
